In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from statsmodels.formula.api import Logit
from sklearn.preprocessing import scale
from sklearn.cross_validation import cross_val_score


READ DATA INTO X AND Y 

In [ ]:
campaign_pd_cleaned = pd.read_csv('../Project2/campaign_pd_cleaned.csv')
campaign_pd_cleaned.dtypes
Y_Total=campaign_pd_cleaned['y']
X_Total=campaign_pd_cleaned.copy()
del X_Total ['y']

SCALE THE VARIABLES

In [ ]:
from sklearn.preprocessing import scale
X_Scaled = scale(X_Total) 

SPLIT INTO TEST TRAIN

In [ ]:
from sklearn.cross_validation import train_test_split

# splits x -> x_train, x_test
#        y -> y_train, y_test
X_train, X_test, Y_train, Y_test = train_test_split(X_Scaled, Y_Total,
                                                    test_size=0.33)

In [10]:
model = LogisticRegression()
model.fit(X_train,Y_train)
y_score=model.predict(X_test)
params=zip(model.coef_.T,X_Total.columns)
params

from sklearn.metrics import roc_curve, auc


fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


ValueError: Can only tuple-index with a MultiIndex

In [8]:
model = LogisticRegression()
RMSE_folds = cross_val_score(model,X_Scaled,Y_Total,
                             cv=4,
                             scoring='roc_auc')
for i,error in enumerate(RMSE_folds):
    print 'Cross val fold %i: %.2f ' % (i, error)

Cross val fold 0: 0.96 
Cross val fold 1: 0.88 
Cross val fold 2: 0.85 
Cross val fold 3: 0.84 


In [ ]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn import datasets
from sklearn.svm import l1_min_c



###############################################################################
# Demo path functions

cs = l1_min_c(X_Scaled, Y_Total, loss='log') * np.logspace(0, 3)


print("Computing regularization path ...")
start = datetime.now()
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
coefs_ = []
for c in cs:
    clf.set_params(C=c)
    clf.fit(X_Scaled, Y_Total)
    coefs_.append(clf.coef_.ravel().copy())
print("This took ", datetime.now() - start)

coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), coefs_)
ymin, ymax = plt.ylim()
plt.xlabel('log(C)')
plt.ylabel('Coefficients')
plt.title('Logistic Regression Path')
plt.axis('tight')
plt.show()

Computing regularization path ...
('This took ', datetime.timedelta(0, 10, 91000))


In [ ]:
clf = linear_model.LogisticRegression(C=1e4)
clf.fit(X_Scaled, y)

# and plot the result
plt.figure(1, figsize=(4, 3))
plt.clf()
plt.scatter(X_train.ravel(), Y_train, color='black', zorder=20)
X_test = np.linspace(-5, 10, 300)


def model(x):
    return 1 / (1 + np.exp(-x))
loss = model(X_test * clf.coef_ + clf.intercept_).ravel()
plt.plot(X_test, loss, color='blue', linewidth=3)

ols = linear_model.LinearRegression()
ols.fit(X_Scaled,Y_Total)
plt.plot(X_test, ols.coef_ * X_test + ols.intercept_, linewidth=1)
plt.axhline(.5, color='.5')

plt.ylabel('y')
plt.xlabel('X')
plt.xticks(())
plt.yticks(())
plt.ylim(-.25, 1.25)
plt.xlim(-4, 10)

plt.show()